# Picture Frame Problem: Formulation 2

A company produces two types of picture frames: each type-1 frame uses 2 hours of labor and 1 ounce of metal and brings $2.25 in profit, and each type-2 frame uses 1 hour of labor and 2 ounces of metal and brings $2.60 in
profit. Each week 4000 labor hours and 5000 ounces of metal are available and the company aims to maximize its weekly profit with an optimal production plan.

To model this problem, let $x_1$ denote the number of type-1 frames produced and $x_2$ denote the number of type-2 frames produced. Then, this can be formulated as the following LP:

\begin{align*}
\underset{x}{\max} \quad &z = 2.25x_1 + 2.6 x_2 \\
\text{s.t.} \ \ \quad &2x_1 + x_2 \leq 4000, \\
&x_1 + 2x_2 \leq 5000, \\
&x \geq 0.
\end{align*}

Let's model this problem using JuMP.

Sometimes, problems are large and span more than just a few variables. Let's see how we would model problems with many variables and constraints.

We can use ranges to represent the amount of variables or constraints for a specific said type. 

For example, if we had multiple y[1], y[2], y[3], we could represent it as follows

In [4]:
using JuMP
using HiGHS


First, let's define our model. 


In [9]:
modelNew = Model(HiGHS.Optimizer)

A JuMP Model
Feasibility problem with:
Variables: 0
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: HiGHS

Let's define multiple variables.

In [10]:
@variable(modelNew, y[1:3] >= 0);

We can also define multiple constriants the same way.

In [12]:
@constraint(modelNew, c[i = 1:2], y[i] <= 2y[i + 1]);


The above code produces:

$y1 <= 2y2$

 
$y2 <= 2y3$

We can also add vectorized constraints:

In [14]:
modelVectorized = Model(HiGHS.Optimizer);
@variable(modelVectorized, x[1:2] >= 0);
A = [1 2; 3 4]; # Creates a 2x2 matrix of the first components over the second
b = [5, 6]; # RHS Vector 
@constraint(modelVectorized, con, A * x .== b); # Notice the dot before the ==, tells Julia all the operations need to be done component wise


Best Practices
- Write code that is separate from the model
- Use dictionaries to make the data more modualar
- Use expressions to write more readable code


Vectors of symbols can be defined as :

In [17]:
picframe2 = Model(HiGHS.Optimizer);
types = [:type_1, :type_2];
labor = Dict(:type_1 => 2, :type_2 => 1);
resources = Dict(:type_1 => 1, :type_2 => 2);
profit = Dict(:type_1 => 2.25, :type_2 => 2.6);

@variable(picframe2, frame[types] >= 0); # types is x1 and x2 
@constraint(picframe2, laborConstraint, sum(labor[i] * frame[i] for i in types) <= 4000) # labor constraint
@constraint(picframe2, resourceConstraint, sum(resources[i] * frame[i] for i in types) <= 5000) # resource constraint

@objective(picframe2, Max, sum(profit[i] * frame[i] for i in types));

print(picframe2)
